##### Emanuele Anarratone 20050533
# Quantum Identity Authentication
## Autenticazione su un canale quantistico per comunicare
### 1. Introduzione

La QKD (Quantum Key Distribution) è un protocollo di crittografia quantistica che consente di comunicare chiavi tra due entità su un canale non sicuro.
I diversi protocolli esistenti di QKD utilizzano la meccanica quantistica per garantire che la trasmissione non venga intercettata tramite l'utilizzo di una coppia di stati EPR (Einstein-Podolsky-Rosen) o stati di Bell la cui misura è correlata solo tra mittente e destinatario reale della comunicazione. 
Se si verifica un'attacco di tipo MITM (Man-In-The-Middle) la misurazione eseguita dall'attaccante distrugge/consuma l'informazione originale, rendendo le coppie di stati **entangled** tra mittente e destinatario disallineate: in questo modo è possibile verificare la presenza di intercettazioni sul canale.
La QIA (Quantum Identity Authentication) è un protocollo che sfrutta gli stati EPR per autenticare le parti coinvolte attraverso firme quantistiche e chiavi di sicurezza definite a priori, al fine di stabilire un canale quantistico autenticato per la successiva comunicazione.

### 2. Panoramica preliminare

La QIA realizzata in questo progetto utilizza gli stati entangled con le successive basi e misurazioni per codificare un HMAC (Hash-based Message Authentication Code) il quale consiste in  un codice di autenticazione del messaggio basato su una funzione di hash crittografica (in questo esempio utilizzerò SHA-256) e una chiave segreta condivisa tra le parti.

Inizialmente, Alice e Bob condividono il canale, per poter comunicare devono prima autenticarsi.
Sia dato per ipotesi un server esterno (il cui comportamento è semplificato in questo progetto a fini d'esempio) il quale apre la connessione iniziale per sincronizzare gli stati di Alice e Bob.


In [ ]:
from qiskit import QuantumCircuit, transpile                                
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session       

import random, hmac, hashlib

In [ ]:
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="TOKEN",
    overwrite=True
)

service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")  #Inizializzazione del backend sulla macchina di Brisbane


C:\Users\Emanuele\AppData\Local\Temp\ipykernel_14596\2435420910.py:7: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService()


In [6]:
# Funzione per HMAC 
def compute_hmac(key: bytes, message: str) -> str:
    return hmac.new(key, message.encode(), hashlib.sha256).hexdigest()


### 3. Scelta della tolleranza e precisione

L'algoritmo genera un numero `rounds` di stati entangled; un valore sufficientemente elevato di `rounds` garantisce una maggiore precisione, a scapito di un maggiore costo computazionale.
Al termine delle misurazioni, il server riceve le firme di Alice e Bob. Il parametro `threshold` rappresenta il limite di tolleranza oltre il quale è possibile considerare l'autenticazione riuscita; valori inferiori a `threshold` comportano la negazione dell'accesso al canale.
La scelta di un valore adeguato per `threshold` deve tenere conto principalmente della tolleranza all’errore dei singoli qubit utilizzati. La macchina **Brisbane**, ad esempio, presenta una tolleranza media (dall'ultima calibrazione effettuata) pari a `1.85e-2`.
Con un numero sufficientemente alto di `rounds`, un valore di `threshold = 0.95` può essere considerato accettabile, considerando che in condizioni ottimali il tasso di correttezza misurato è solitamente superiore al 98%.


In [7]:
rounds = 1000
threshold = 0.95

alice_basis, bob_basis = [], []
alice_results, bob_results = [], []
circuits = []   #Per memorizzare i circuiti ed eseguire il sampling in un'unica chiamata

### 4. Circuito

L'applicazione delle porte **Hadamard (H)** e **CNOT** a un circuito a due qubit genera uno **stato di Bell**, ovvero uno stato entangled.
Alice e Bob scelgono casualmente la **base di misura** da utilizzare: la base Z (computazionale) oppure la base X (ottenuta tramite trasformazione di Hadamard).
Successivamente, viene eseguita la misura e vengono salvati:
- le basi utilizzate, che verranno impiegate per la generazione dell'HMAC;
- il circuito eseguito, da fornire al `sampler` per ottenere i risultati delle misurazioni.

Il procedimento è similare a quello per la QKD: 
- Alice ha un messaggio (es. `10111010101101010`) e sceglie **casualmente** le basi che utilizzerà (es. `XZZZXZXXXZXZXXXZZZXZ`)
- Alice dopodichè codifica messaggio e basi e ottiene una serie di qubit (es. `|1>|1>|+>|0>|->|0>|->|0>|->|1>|+>|->|+>|1>|+>|0>`) che invierà a Bob
- Bob riceve i qubit e decide le basi con cui misurare (es. `XZZZXZXZXZXZZZXZ`)

Se Alice e Bob per **lo stesso Qubit** utilizzano **la stessa base** avranno risultati correlati, quando lo stato è entangled e le parti sono autentiche la correlazione è sicura (>98%)

In [8]:
for i in range(rounds):
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)

    alice_b = random.choice(['Z', 'X'])
    bob_b = random.choice(['Z', 'X'])

    if alice_b == 'X':  #Applicazione Hadamard di Alice
        qc.h(0)
    
    if bob_b == 'X':    #Applicazione Hadamard di Bob
        qc.h(1)
    
    qc.measure([0, 1], [0, 1])  
    circuits.append(qc)         
    alice_basis.append(alice_b)     
    bob_basis.append(bob_b)     #Le basi di Alice e Bob vengono salvate per il calcolo dell'HMAC


### 5. Esecuzione

Su una macchina reale, come quella di Brisbane, i risultati vengono restituiti come distribuzioni di probabilità, le quali convergono (con quasi certezza matematica) verso un determinato valore atteso in fase di misurazione.

Attraverso un ciclo `for`, vengono estratte le **rounds** misurazioni contenute in `results._pub_results` e trasformate in bitstring, così da poter accedere ai singoli bit relativi alle misurazioni effettuate da Alice e Bob.

> **Nota**: l'utilizzo di `_pub_results` non è formalmente corretto, in quanto si tratta di un attributo **privato** della classe `SamplerPubResult`. Il metodo pubblico `quasi_dists[]` è quello ufficialmente previsto per accedere ai dati, ma al momento dello sviluppo, a causa di **deprecazioni delle API**, il suo utilizzo ha causato diversi problemi tecnici, impedendone l'adozione.


In [9]:
with Session(backend=backend) as session:
    sampler = Sampler(mode=backend)
    t_circuits = transpile(circuits, backend=backend)
    job = sampler.run(t_circuits, shots=1)
    results = job.result()

    for i, bitarray in enumerate(results._pub_results):
        bitstring = bitarray.data.c.get_bitstrings()[0]
        alice_results.append(bitstring[0])
        bob_results.append(bitstring[1])

base_primitive._get_mode_service_backend:WARNING:2025-06-11 17:09:50,801: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
c:\Users\Emanuele\miniconda3\envs\qc\Lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:1129: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)


### 6. Invio delle firme

Dopo l’esecuzione delle misurazioni e la computazione dei risultati, questi vengono inviati al server per poter accedere al canale autenticato.

Alice e Bob generano ciascuno il proprio **HMAC** calcolato su un messaggio che include:
- i risultati delle misurazioni,
- le basi utilizzate per la misura,
- e la chiave segreta assegnata loro in fase di inizializzazione.

Il server utilizzerà questi HMAC per verificare l'autenticità delle parti e autorizzare l'accesso al canale solo se le firme risultano valide.


In [10]:
K_A = b'chiavedicomunicazione' #semplificazioni d'esempio, possono essere diverse tra loro o randomizzate (OTP)
K_B = b'chiavedicomunicazione'


msg_A = ''.join(alice_basis) + ''.join(alice_results)
msg_B = ''.join(bob_basis) + ''.join(bob_results)
hmac_A = compute_hmac(K_A, msg_A)
hmac_B = compute_hmac(K_B, msg_B)
#L'invio dei messaggi e degli HMAC non è implementato in questo esempio, ma dovrebbe essere fatto in un canale affidabile

### 7. Server

Il server riceve le firme (HMAC) da Alice e Bob e ne verifica la validità confrontandole con quelle attese, calcolate localmente sulla base dei dati ricevuti e delle chiavi segrete assegnate.

Successivamente, verifica la **correlazione** tra le misurazioni effettuate da Alice e Bob.  
Se la percentuale di corrispondenza supera o equivale al valore soglia definito da `threshold`, l’autenticazione viene considerata riuscita e l’accesso al canale viene autorizzato.  
In caso contrario, la connessione viene rifiutata.
In questo modo, è possibile quindi definire un protocollo per l'autenticazione ad un canale quantistico tramite l'utilizzo di firme quantistiche e degli stati entangled condivisi per aprire una connessione.


In [ ]:
valid_A = (compute_hmac(K_A, msg_A) == hmac_A)
valid_B = (compute_hmac(K_B, msg_B) == hmac_B)

used, matches = 0, 0
for a_b, b_b, a_r, b_r in zip(alice_basis, bob_basis, alice_results, bob_results):
    if a_b == b_b:
        used += 1
        if a_r == b_r:
            matches += 1

match_ratio = matches / used if used else 0
authenticated = valid_A and valid_B and match_ratio >= threshold

print(f"""
        Bits usati: {used}
        Coincidenze: {matches}
        Match %: {match_ratio*100:.2f}%
        HMAC Alice valido: {valid_A}
        HMAC Bob valido: {valid_B}
        Autenticazione {"riuscita" if authenticated else "non riuscita"}
      """
      )



      "Bits usati: 474
        Coincidenze: 465
        Match %: 98.10%
        HMAC Alice valido: True
        HMAC Bob valido: True
        Autenticazione riuscita
      


### 8. Analisi dell'output – Conclusioni
Dall’output ottenuto si osserva che, su 1000 round, 474 vengono effettivamente utilizzati. Questo è dovuto al fatto che Alice e Bob hanno una probabilità del 50% di scegliere la stessa base per la misurazione; ci si aspetta quindi che i bit utilizzati siano distribuiti secondo una distribuzione binomiale con valore atteso pari al 50% del totale.
Dei 474 bit utilizzati, 465 risultano coincidenti. Le discrepanze sono attribuibili all’esecuzione del programma su una macchina reale, dove errori casuali dovuti a rumore o interferenze quantistiche possono causare discrepanze nei bit misurati. Una scelta oculata del parametro threshold consente comunque di accettare l’autenticazione anche in presenza di un numero contenuto di errori.
Poiché gli HMAC di Alice e Bob risultano validi e i loro risultati coincidono nel 98% dei casi, si conclude che l’autenticazione è avvenuta con successo, e che Alice e Bob possono procedere a comunicare in sicurezza attraverso il canale.
